In [4]:
import pandas as pd
from sqlalchemy import create_engine
import numpy as np
import time
from sodapy import Socrata

In [27]:
#json file
crime_df = pd.DataFrame()
for i in range(500):
    offset = 2000*i
    limit = 2000
    df = pd.read_json(f"https://data.cityofchicago.org/resource/crimes.json?$limit=2000&$offset={offset}")
    crime_df = crime_df.append(other=df)

In [33]:
#selection of data from main table
new_crime_df = crime_df[['ward','primary_type','arrest','location_description','year','latitude','longitude']].copy()
new_crime_df.head()

,ward,primary_type,arrest,location_description,year,latitude,longitude
0,16.0,THEFT,False,SIDEWALK,2019,41.771737,-87.664006
1,28.0,NARCOTICS,True,STREET,2019,41.879612,-87.709447
2,24.0,INTERFERENCE WITH PUBLIC OFFICER,True,POLICE FACILITY/VEH PARKING LOT,2019,41.873699,-87.704705
3,24.0,BATTERY,True,STREET,2019,41.865495,-87.728818
4,17.0,ROBBERY,False,GAS STATION,2019,41.771532,-87.702965


In [57]:
#transform dataframe
transf_crime_df = ['ward','primary_type','arrest','year','location_description','latitude','longitude']
transf_crime_chicago = new_crime_df[transf_crime_df].copy()

In [58]:
# Rename the column headers
transf_crime_chicago = transf_crime_chicago.rename(columns={"ward": "ward_no",
                                                          "primary_type": "Crimetype",
                                                          "arrest": "Arrest",
                                                           "year":"Year",
                                                            "location_description":"Crime_location",
                                                            "latitude":"Latitude",
                                                            "longitude":"Longitude"})

In [59]:
transf_crime_chicago.head()

,ward_no,Crimetype,Arrest,Year,Crime_location,Latitude,Longitude
0,16.0,THEFT,False,2019,SIDEWALK,41.771737,-87.664006
1,28.0,NARCOTICS,True,2019,STREET,41.879612,-87.709447
2,24.0,INTERFERENCE WITH PUBLIC OFFICER,True,2019,POLICE FACILITY/VEH PARKING LOT,41.873699,-87.704705
3,24.0,BATTERY,True,2019,STREET,41.865495,-87.728818
4,17.0,ROBBERY,False,2019,GAS STATION,41.771532,-87.702965


In [60]:
transf_crime_chicago.shape


(1000000, 7)

In [73]:
#connection with database name "chicago_crime_db"
rds_connection_string = f"postgres:sherkhan@localhost:5432/chicago_crime_db"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [66]:
#inserting and creating table in database
transf_crime_chicago.to_sql(name='chicago_crimes', con=engine, if_exists='append', index=False)

In [57]:
#reading data from database
df = pd.read_sql_query('select * from chicago_crimes', con=engine)


In [35]:
#changing name of coulms from previous table 
df.columns = ['ward_no', 'crime_type', 'arrest', 'year', 'crime_location', 'latitude',
       'longitude']

In [36]:
df.shape

(1000000, 7)

In [25]:
df_crime = df
df_crime.head()

,ward_no,crime_type,arrest,year,crime_location,latitude,longitude
0,16.0,THEFT,False,2019,SIDEWALK,41.771737,-87.664006
1,28.0,NARCOTICS,True,2019,STREET,41.879612,-87.709447
2,24.0,INTERFERENCE WITH PUBLIC OFFICER,True,2019,POLICE FACILITY/VEH PARKING LOT,41.873699,-87.704705
3,24.0,BATTERY,True,2019,STREET,41.865495,-87.728818
4,17.0,ROBBERY,False,2019,GAS STATION,41.771532,-87.702965


In [5]:
#connection with database name "chicago_crime_db"
rds_connection_string = f"postgres:sherkhan@localhost:5432/chicago_crime_db"
engine = create_engine(f'postgresql://{rds_connection_string}')


In [ ]:
#inserting and creating table in database with table name chicago_crime
df_crime.to_sql(name='chicago_crime', con=engine, if_exists='append', index=False)


In [23]:
#retriving data from database
pd.read_sql_query('select * from chicago_crime', con=engine).head()

,ward_no,crime_type,arrest,year,crime_location,latitude,longitude
0,16.0,THEFT,False,2019,SIDEWALK,41.771737,-87.664006
1,28.0,NARCOTICS,True,2019,STREET,41.879612,-87.709447
2,24.0,INTERFERENCE WITH PUBLIC OFFICER,True,2019,POLICE FACILITY/VEH PARKING LOT,41.873699,-87.704705
3,24.0,BATTERY,True,2019,STREET,41.865495,-87.728818
4,17.0,ROBBERY,False,2019,GAS STATION,41.771532,-87.702965


In [ ]:
#By using Chicago Crime API we retrive data and save in database "chicago_crime_db" table name "chciago_crime"
# we export the cvs file from database chciago_crime
# will use that table 


In [38]:
#define json file chicago_econamy
json_file = "https://data.cityofchicago.org/resource/kn9c-c2s2.json"
chicago_socialecnom_df = pd.read_json(json_file)
#resourses https://data.cityofchicago.org/Health-Human-Services/Census-Data-Selected-socioeconomic-indicators-in-C/kn9c-c2s2

In [39]:
chicago_socialecnom_df.head()

,ca,community_area_name,hardship_index,per_capita_income_,percent_aged_16_unemployed,percent_aged_25_without_high_school_diploma,percent_aged_under_18_or_over_64,percent_households_below_poverty,percent_of_housing_crowded
0,1.0,Rogers Park,39.0,23939,8.7,18.2,27.5,23.6,7.7
1,2.0,West Ridge,46.0,23040,8.8,20.8,38.5,17.2,7.8
2,3.0,Uptown,20.0,35787,8.9,11.8,22.2,24.0,3.8
3,4.0,Lincoln Square,17.0,37524,8.2,13.4,25.5,10.9,3.4
4,5.0,North Center,6.0,57123,5.2,4.5,26.2,7.5,0.3


In [40]:
#selection of attribute from main table
new_chicago_soc_df = chicago_socialecnom_df[['ca','community_area_name','per_capita_income_','percent_aged_16_unemployed','percent_aged_25_without_high_school_diploma','percent_aged_under_18_or_over_64','percent_households_below_poverty','percent_of_housing_crowded']].copy()
new_chicago_soc_df.head()

,ca,community_area_name,per_capita_income_,percent_aged_16_unemployed,percent_aged_25_without_high_school_diploma,percent_aged_under_18_or_over_64,percent_households_below_poverty,percent_of_housing_crowded
0,1.0,Rogers Park,23939,8.7,18.2,27.5,23.6,7.7
1,2.0,West Ridge,23040,8.8,20.8,38.5,17.2,7.8
2,3.0,Uptown,35787,8.9,11.8,22.2,24.0,3.8
3,4.0,Lincoln Square,37524,8.2,13.4,25.5,10.9,3.4
4,5.0,North Center,57123,5.2,4.5,26.2,7.5,0.3


In [43]:
new_chicago_soc_df.shape

(78, 8)

In [45]:
#transform dataframe
transf_chic_soc = ['ca','community_area_name','per_capita_income_','percent_aged_16_unemployed','percent_aged_25_without_high_school_diploma','percent_aged_under_18_or_over_64','percent_households_below_poverty','percent_of_housing_crowded']
transfer_chicago_spc = new_chicago_soc_df[transf_chic_soc].copy()

In [46]:
#rename the column header
transfer_chicago_spc = transfer_chicago_spc.rename(columns={"ca":"ward_no",
                                                           "community_area_name":"Areaname",
                                                           "per_capita_income_":"income_per_capital",
                                                           "percent_aged_16_unemployed":"sixteen_plus_unemployed",
                                                           "percent_aged_25_without_high_school_diploma":"twenty_plus_without_diploma",
                                                           "percent_aged_under_18_or_over_64":"population_eighteen_sixtyfour",
                                                           "percent_households_below_poverty":"population_below_poverty",
                                                           "percent_of_housing_crowded":"overpopulation"})
transfer_chicago_spc.head()


,ward_no,Areaname,income_per_capital,sixteen_plus_unemployed,twenty_plus_without_diploma,population_eighteen_sixtyfour,population_below_poverty,overpopulation
0,1.0,Rogers Park,23939,8.7,18.2,27.5,23.6,7.7
1,2.0,West Ridge,23040,8.8,20.8,38.5,17.2,7.8
2,3.0,Uptown,35787,8.9,11.8,22.2,24.0,3.8
3,4.0,Lincoln Square,37524,8.2,13.4,25.5,10.9,3.4
4,5.0,North Center,57123,5.2,4.5,26.2,7.5,0.3


In [48]:
#save our data in database table name  "chicago_economy"
transfer_chicago_spc.to_sql(name='chicago_economy', con=engine, if_exists='append', index=False)

In [53]:
#retriving data from database
pd.read_sql_query('select * from chicago_economy', con=engine).head()

,ward_no,Areaname,income_per_capital,sixteen_plus_unemployed,twenty_plus_without_diploma,population_eighteen_sixtyfour,population_below_poverty,overpopulation
0,1.0,Rogers Park,23939,8.7,18.2,27.5,23.6,7.7
1,2.0,West Ridge,23040,8.8,20.8,38.5,17.2,7.8
2,3.0,Uptown,35787,8.9,11.8,22.2,24.0,3.8
3,4.0,Lincoln Square,37524,8.2,13.4,25.5,10.9,3.4
4,5.0,North Center,57123,5.2,4.5,26.2,7.5,0.3


In [49]:
area_df = ['ca','community_area_name']
area_code_df = chicago_socialecnom_df[area_df].copy()

In [51]:
#rename the column header
area_code_name = area_code_df.rename(columns={"ca":"ward_no",
                                    "community_area_name":"Areaname"})
area_code_name.head()

,ward_no,Areaname
0,1.0,Rogers Park
1,2.0,West Ridge
2,3.0,Uptown
3,4.0,Lincoln Square
4,5.0,North Center


In [55]:
#save our data in database table name  "area_name"
area_code_name.to_sql(name='area_name', con=engine, if_exists='append', index=False)

In [68]:
#retriving data from database
df = pd.read_sql_query('select * from area_name', con=engine)


,ward_no,Areaname
0,1.0,Rogers Park
1,2.0,West Ridge
2,3.0,Uptown
3,4.0,Lincoln Square
4,5.0,North Center


In [77]:
main_df = pd.merge(df, df_crime,how="inner", on="ward_no")
main_df = data_df
data_df.head()

,ward_no,Areaname,crime_type,arrest,year,crime_location,latitude,longitude
0,1.0,Rogers Park,BATTERY,False,2019,NURSING HOME/RETIREMENT HOME,41.907032,-87.696980
1,1.0,Rogers Park,BATTERY,False,2019,ALLEY,41.907417,-87.689603
2,1.0,Rogers Park,BATTERY,False,2019,CHA APARTMENT,41.894337,-87.661706
3,1.0,Rogers Park,THEFT,False,2019,DEPARTMENT STORE,41.909552,-87.676179
4,1.0,Rogers Park,OTHER OFFENSE,False,2019,RESIDENCE,41.900377,-87.671083


In [ ]:
#save our data in database table name  "chicago_data"
data_df.to_sql(name='chicago_data', con=engine, if_exists='replace', index=False, chunksize=10000)

In [84]:
df = data_df.head(1000)
df.to_sql(name='chicago_data', con=engine, if_exists='replace', index=False)

In [ ]:
pd.read_sql_query('select * from chicago_data', con=engine).head()

In [ ]:
#retriving data from database
pd.read_sql_query('select * from chicago_economy', con=engine).head()

In [89]:
data_df.head()

,ward_no,Areaname,crime_type,arrest,year,crime_location,latitude,longitude
0,1.0,Rogers Park,BATTERY,False,2019,NURSING HOME/RETIREMENT HOME,41.907032,-87.696980
1,1.0,Rogers Park,BATTERY,False,2019,ALLEY,41.907417,-87.689603
2,1.0,Rogers Park,BATTERY,False,2019,CHA APARTMENT,41.894337,-87.661706
3,1.0,Rogers Park,THEFT,False,2019,DEPARTMENT STORE,41.909552,-87.676179
4,1.0,Rogers Park,OTHER OFFENSE,False,2019,RESIDENCE,41.900377,-87.671083


In [ ]:
df_year = data_df.groupby("year")
crime_2015 = df_year.get_group(2015)

In [ ]:
crime_2015.head()

In [96]:
crime_2015.to_sql(name='crime_final', con=engine, if_exists='append', index=False)

In [9]:
df_area = pd.read_sql_query('select * from crime_final', con=engine).head()
df_area

,ward_no,Areaname,crime_type,arrest,year,crime_location,latitude,longitude
0,1.0,Rogers Park,BURGLARY,False,2015,APARTMENT,41.895733,-87.687457
1,1.0,Rogers Park,DECEPTIVE PRACTICE,False,2015,OTHER,41.909098,-87.668807
2,1.0,Rogers Park,OTHER OFFENSE,False,2015,WAREHOUSE,41.904501,-87.668263
3,1.0,Rogers Park,CRIMINAL DAMAGE,False,2015,STREET,41.907127,-87.678210
4,1.0,Rogers Park,CRIMINAL DAMAGE,False,2015,STREET,41.912757,-87.698087


In [10]:
df_area.dtypes

ward_no           float64
Areaname           object
crime_type         object
arrest               bool
year                int64
crime_location     object
latitude          float64
longitude         float64
dtype: object

In [25]:
#selection of data from main table
df_new_tb = df_area[['ward_no','Areaname','latitude','longitude']].copy()
df_new_tb.head()

,ward_no,Areaname,latitude,longitude
0,1.0,Rogers Park,41.895733,-87.687457
1,1.0,Rogers Park,41.909098,-87.668807
2,1.0,Rogers Park,41.904501,-87.668263
3,1.0,Rogers Park,41.907127,-87.678210
4,1.0,Rogers Park,41.912757,-87.698087


In [26]:
#save table in db
df_new_tb.to_sql(name='data_log', con=engine, if_exists='append', index=False)

In [27]:
pd.read_sql_query('select * from data_log', con=engine).head()


,ward_no,Areaname,latitude,longitude
0,1.0,Rogers Park,41.895733,-87.687457
1,1.0,Rogers Park,41.909098,-87.668807
2,1.0,Rogers Park,41.904501,-87.668263
3,1.0,Rogers Park,41.907127,-87.678210
4,1.0,Rogers Park,41.912757,-87.698087


In [ ]:
#filter crime data ob year as per requirement
df_year = data_df.groupby("year")
crime_2015 = df_year.get_group(2015)


In [ ]:
crime_2015.head()

In [ ]:
#count crime per area
df_data1 = crime_2015.groupby("Areaname").count() 
df_new = df_data1

In [ ]:
df_new.head()

In [ ]:
crime_accident = df_new[['ward_no']].copy()
crime_accident.head()

In [ ]:
#renaming
crime_accident = crime_accident.rename(columns={"ca":"ward_no",
                                                           "ward_no":"crime_no",})
crime_accident.head()

In [ ]:
#save the table in database
crime_accident.to_sql(name='accident', con=engine, if_exists='append', index=False)

In [ ]:
#merg no of accident and chicago economy
new1_df = pd.merge(crime_accident, transfer_chicago_spc,how="inner", on="Areaname")
data5_df = new1_df
data5_df.head()

In [29]:
#retriving data feom database
merg_df = pd.read_sql_query('select * from merg_dt', con=engine)
merg_df.head()

,Areaname,crime_no,ward_no,income_per_capital,sixteen_plus_unemployed,twenty_plus_without_diploma,population_eighteen_sixtyfour,population_below_poverty,overpopulation
0,Albany Park,194,14.0,21323,10.0,32.9,32.0,19.2,11.3
1,Armour Square,314,34.0,16148,16.7,34.5,38.3,40.1,5.7
2,Austin,229,25.0,15957,22.6,24.4,37.9,28.6,6.3
3,Avalon Park,151,45.0,24454,21.1,10.6,39.3,17.2,1.4
4,Avondale,421,21.0,20039,9.2,24.7,31.0,15.3,6.0
